In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import json
import osmnx as ox

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

geo

---

In [1]:
# FIXME libcity 不需要 line_graph !!!
# 以下 .geo .rel 都是错的

In [10]:
graph_drive=nx.read_gpickle("../data/futian_graph_drive.pkl")
line_graph_drive=nx.read_gpickle("../data/futian_line_graph_drive.pkl")

list(line_graph_drive.nodes)[:10]

[(296024760, 6033481369, 0),
 (6033481369, 5435020958, 0),
 (409392019, 6033481329, 0),
 (6033481329, 7959990534, 0),
 (6033481329, 7959990557, 0),
 (499236929, 499264828, 0),
 (499264828, 499264853, 0),
 (499237442, 499256522, 0),
 (499256522, 499374765, 0),
 (499237442, 7232025523, 0)]

In [11]:
df_roads=pd.DataFrame()

df_roads["geo_id"]=line_graph_drive.nodes
df_roads=df_roads.reset_index(drop=True)

df_roads["type"]="LineString"

df_roads["coordinates"]=None

df_roads

,geo_id,type,coordinates
0,"(296024760, 6033481369, 0)",LineString,None
1,"(6033481369, 5435020958, 0)",LineString,None
2,"(409392019, 6033481329, 0)",LineString,None
3,"(6033481329, 7959990534, 0)",LineString,None
4,"(6033481329, 7959990557, 0)",LineString,None
...,...,...,...
403,"(8869146293, 7959990521, 0)",LineString,None
404,"(8516154912, 8516154910, 0)",LineString,None
405,"(8516154910, 8516154909, 0)",LineString,None
406,"(8516154910, 8875446876, 0)",LineString,None


In [12]:
def get_coords(row):
    road=row["geo_id"]

    road_data_dict=graph_drive.get_edge_data(road[0], road[1])[road[2]]

    if "geometry" not in road_data_dict:
        return row

    row["coordinates"]=list(road_data_dict["geometry"].coords)

    return row

df_roads=df_roads.apply(get_coords, axis=1)

df_roads
df_roads.isna().sum()

,geo_id,type,coordinates
0,"(296024760, 6033481369, 0)",LineString,"[(114.0592761, 22.5315228), (114.0597732, 22.5..."
1,"(6033481369, 5435020958, 0)",LineString,None
2,"(409392019, 6033481329, 0)",LineString,None
3,"(6033481329, 7959990534, 0)",LineString,None
4,"(6033481329, 7959990557, 0)",LineString,"[(114.0608672, 22.5318501), (114.0607746, 22.5..."
...,...,...,...
403,"(8869146293, 7959990521, 0)",LineString,"[(114.0629074, 22.5338432), (114.0627946, 22.5..."
404,"(8516154912, 8516154910, 0)",LineString,None
405,"(8516154910, 8516154909, 0)",LineString,None
406,"(8516154910, 8875446876, 0)",LineString,"[(114.0630009, 22.537682), (114.062999, 22.537..."


geo_id           0
type             0
coordinates    180
dtype: int64

In [13]:
df_roads=df_roads.reset_index(drop=True)
df_roads["geo_id"]=df_roads.index
df_roads=df_roads.fillna("[()]")
# df_roads=df_roads.dropna()

df_roads

,geo_id,type,coordinates
0,0,LineString,"[(114.0592761, 22.5315228), (114.0597732, 22.5..."
1,1,LineString,[()]
2,2,LineString,[()]
3,3,LineString,[()]
4,4,LineString,"[(114.0608672, 22.5318501), (114.0607746, 22.5..."
...,...,...,...
403,403,LineString,"[(114.0629074, 22.5338432), (114.0627946, 22.5..."
404,404,LineString,[()]
405,405,LineString,[()]
406,406,LineString,"[(114.0630009, 22.537682), (114.062999, 22.537..."


In [15]:
# df_roads.to_csv("../data/libcity_mm/sz_taxi_mm.geo", index=False)

rel

---

In [16]:
adj=nx.convert_matrix.to_numpy_array(line_graph_drive)

adj

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
rel=[]
rel_id_counter=0

for i in range(len(adj)):
    for j in range(len(adj[0])):
        if adj[i][j]:
            rel.append([rel_id_counter, "geo", i, j])
            rel_id_counter+=1

df_rel=pd.DataFrame(rel, columns=["rel_id", "type", "origin_id", "destination_id"])

df_rel

,rel_id,type,origin_id,destination_id
0,0,geo,0,1
1,1,geo,1,130
2,2,geo,2,3
3,3,geo,2,4
4,4,geo,3,142
...,...,...,...,...
682,682,geo,404,406
683,683,geo,405,156
684,684,geo,405,157
685,685,geo,406,407


In [18]:
# df_rel.to_csv("../data/libcity_mm/sz_taxi_mm.rel", index=False)

usr

---

In [5]:
taxi_dict={}
taxi_counter=0

for taxi_file in os.listdir("../data/taxi_after_proc/tracks"):
    taxi_license=taxi_file.split("_")[1].split(".")[0]
    if taxi_license not in taxi_dict:
        taxi_dict[taxi_license]=taxi_counter
        taxi_counter+=1
        
len(taxi_dict)

20868

In [ ]:
df_usr=pd.DataFrame()
df_usr["usr_id"]=taxi_dict.values()
df_usr["taxi_license"]=taxi_dict.keys()
df_usr

In [21]:
# df_usr.to_csv("../data/libcity_mm/sz_taxi_mm.usr", index=False)

dyna

---

In [ ]:
# 12-02_粤B0C2R7.csv

In [3]:
minlat=22.5310
maxlat=22.5397
minlng=114.0442
maxlng=114.0633

def contains(lat, lng):
    return lat>=minlat and lat<=maxlat and lng>=minlng and lng<=maxlng

In [6]:
# Mon. to Fri.
dyna_file=open("../data/libcity_mm/sz_taxi_mm_filtered.dyna", "w")
write_length=dyna_file.write("dyna_id,type,time,entity_id,traj_id,coordinates\n") # disable printing return value

timedelta_30=pd.Timedelta(seconds=30)
timedelta_600=pd.Timedelta(seconds=600)
    
dyna_id_counter=0

for taxi_file in tqdm(sorted(os.listdir("../data/taxi_after_proc/merged"))):
    date=int(taxi_file.split("_")[0].split("-")[1])
    if date<2 or date>6:
        continue
    if os.path.getsize(f"../data/taxi_after_proc/merged/{taxi_file}")<100:
        continue
    taxi_license=taxi_file.split("_")[1].split(".")[0]
    df_taxi=pd.read_csv(f"../data/taxi_after_proc/merged/{taxi_file}", parse_dates=["gps_time"])
    if df_taxi.empty:
        continue
    
    traj_counter=0
    line_buffer=[]
    last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-600)
    for row in df_taxi.itertuples():
        if not contains(row[1], row[2]):
            continue
        if row[3]-last_time<timedelta_30: # resample: drop <30s
            continue
        if row[3]-last_time>timedelta_600:
            if len(line_buffer)>1: # only store length>1 traj
                write_length=dyna_file.write("".join(line_buffer))
                traj_counter+=1
            line_buffer=[]
            
        last_time=row[3]
        
        line_buffer.append(f"{dyna_id_counter},trajectory,{row[3].strftime('%Y-%m-%dT%H:%M:%SZ')},{taxi_dict[taxi_license]},{traj_counter},\"{[row[2], row[1]]}\"\n")
        dyna_id_counter+=1
        
dyna_file.close()

100%|██████████| 252072/252072 [08:03<00:00, 521.66it/s]   


In [4]:
# @deprecated

# Mon. to Fri.
dyna_file=open("../data/libcity_mm/sz_taxi_mm.dyna", "w")
write_length=dyna_file.write("dyna_id,type,time,entity_id,traj_id,coordinates\n") # disable printing return value

timedelta_30=pd.Timedelta(seconds=30)
timedelta_600=pd.Timedelta(seconds=600)

dyna_id_counter=0

for taxi_file in tqdm(sorted(os.listdir("../data/taxi_after_proc/merged"))):
    date=int(taxi_file.split("_")[0].split("-")[1])
    if date<2 or date>6:
        continue
    if os.path.getsize(f"../data/taxi_after_proc/merged/{taxi_file}")<100:
        continue
    taxi_license=taxi_file.split("_")[1].split(".")[0]
    df_taxi=pd.read_csv(f"../data/taxi_after_proc/merged/{taxi_file}", parse_dates=["gps_time"])
    if df_taxi.empty:
        continue
    
    traj_counter=0
    last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-600)
    for row in df_taxi.itertuples():
        if row[3]-last_time<timedelta_30: # resample: drop <30s
            continue
        if row[3]-last_time>timedelta_600:
            traj_counter+=1
        last_time=row[3]
        
        write_length=dyna_file.write(f"{dyna_id_counter},trajectory,{row[3].strftime('%Y-%m-%dT%H:%M:%SZ')},{taxi_dict[taxi_license]},{traj_counter},\"{[row[2], row[1]]}\"\n")
        dyna_id_counter+=1
        
dyna_file.close()

100%|██████████| 252072/252072 [39:28<00:00, 106.44it/s]   


In [5]:
df_dyna=pd.read_csv("../data/libcity_mm/sz_taxi_mm.dyna", nrows=100)

df_dyna

,dyna_id,type,time,entity_id,traj_id,coordinates
0,0,trajectory,2019-12-02T00:10:30Z,15876,0,"[114.06776, 22.550152]"
1,1,trajectory,2019-12-02T00:11:32Z,15876,0,"[114.06859, 22.54198]"
2,2,trajectory,2019-12-02T00:12:12Z,15876,0,"[114.07125, 22.542738]"
3,3,trajectory,2019-12-02T00:51:17Z,15876,1,"[114.04781, 22.539036]"
4,4,trajectory,2019-12-02T00:51:47Z,15876,1,"[114.04774, 22.538887]"
...,...,...,...,...,...,...
95,95,trajectory,2019-12-02T15:43:25Z,15876,7,"[114.05783, 22.531305]"
96,96,trajectory,2019-12-02T15:51:55Z,15876,7,"[114.05137, 22.53659]"
97,97,trajectory,2019-12-02T15:55:40Z,15876,7,"[114.05117, 22.53749]"
98,98,trajectory,2019-12-02T15:57:38Z,15876,7,"[114.051216, 22.539156]"


config

---

In [7]:
import json

config={}

config["geo"]={}
config["geo"]["including_types"]=["LineString"]
config["geo"]["LineString"]={}

config["rel"]={}
config["rel"]["including_types"]=["geo"]
config["rel"]["geo"]={}

config["usr"]={}
config["usr"]["properties"]={"taxi_license": "str"}

config["dyna"]={}
config["dyna"]["including_types"]=["trajectory"]
config["dyna"]["trajectory"]={"time": "time", "entity_id": "usr_id", "traj_id": "num", "coordinates": "coordinate"}

config["info"]={}
# config["info"]["data_files"]="sz_taxi_mm_filtered"
config["info"]["data_files"]="sz_taxi_mm_test"
config["info"]["geo_file"]="sz_taxi_mm"
config["info"]["rel_file"]="sz_taxi_mm"
config["info"]["init_weight_inf_or_zero"]="zero" # adj matrix not connected: 0 (inf: infinity)
config["info"]["set_weight_link_or_dist"]="link" # adj matrix 01 (dist: use weight)
config["info"]["calculate_weight_adj"]=False
# config["info"]["weight_adj_epsilon"]=0.1 # disabled when the above is false

json.dump(config, open("../data/libcity_mm/config.json", "w", encoding="utf-8"), ensure_ascii=False)
config

{'geo': {'including_types': ['LineString'], 'LineString': {}},
 'rel': {'including_types': ['geo'], 'geo': {}},
 'usr': {'properties': {'taxi_license': 'str'}},
 'dyna': {'including_types': ['trajectory'],
  'trajectory': {'time': 'time',
   'entity_id': 'usr_id',
   'traj_id': 'num',
   'coordinates': 'coordinate'}},
 'info': {'data_files': 'sz_taxi_mm_test',
  'geo_file': 'sz_taxi_mm',
  'rel_file': 'sz_taxi_mm',
  'init_weight_inf_or_zero': 'zero',
  'set_weight_link_or_dist': 'link',
  'calculate_weight_adj': False}}

In [ ]:
# nohup python run_model.py --task map_matching --model HMMM --dataset sz_taxi_mm > mm.log 2>&1 &